# *RUN THE FIRST TWO CHUNKS*

In [1]:
import os
import pandas as pd
import numpy as np
from tqdm.notebook import tqdm
from sklearn.metrics import r2_score

#### Import LOB, LOD, LOQ values

In [2]:
rpath = "C:\\Users\\ytsma22c\\Google Drive\\Misc Work\\LOD paper\\LOD_calcs.csv"
results = pd.read_csv(rpath)

# First/reset calculations?
#### Set up results table

In [3]:
full_table = pd.DataFrame({
    "element" : [],
    'instrument' : [],
    'atmosphere' : [],
    'num_range' : [],
    "Avg_Braga" : [],
    "Avg_metals" : [],
    "RMSEP_Braga" : [],
    "RMSEP_metals" : [],
    "R2_Braga" : [],
    "R2_metals" : []
})

# Adding new data?
#### Import existing table

In [ ]:
fpath = "C:\\Users\\ytsma22c\\Google Drive\\Misc Work\\LOD paper\\test_calcs.csv"
full_table = pd.read_csv(fpath)

# \*\*\*Start here with a new calculation\*\*\*
Data should be in a `XX_preds.csv` or `XX_predictions.csv` format within folders separated by random number region (0-750 or 250-1000)

- **LANL 0-750:** C:\\Users\\ytsma22c\\Google Drive\\Misc Work\\LOD paper\\LANL calculations\\models\\0-750\\
- **LANL 250-1000:** C:\\Users\\ytsma22c\\Google Drive\\Misc Work\\LOD paper\\LANL calculations\\models\\250-1000\\

In [5]:
instrument = input("What instrument was the data collected on? ")
atm = input("What atmosphere was the data collected under? ")
num_range = input("What range of random numbers were the models trained on? (0-750 or 250-1000)")
folder = input('Folder path containing predictions:')
predlist = os.listdir(folder)

What instrument was the data collected on? LANL
What atmosphere was the data collected under? Mars
What range of random numbers were the models trained on? (0-750 or 250-1000)250-1000
Folder path containing predictions:C:\Users\ytsma22c\Google Drive\Misc Work\LOD paper\LANL calculations\models\250-1000\


In [6]:
# subselect the relevant parts of the results table
new_results = results[(results.instrument == instrument) & 
                      (results.atmosphere == atm) &
                      (results.num_range == num_range)]

In [7]:
# prepare lists to be filled
# ADD LISTS FOR 'ALL' AND 'ABOVE LOD' CALCULATIONS IF DESIRED
elem = []
avg_braga = []
rmsep_braga = []
r2_braga = []
avg_metal = []
rmsep_metal = []
r2_metal = []

In [8]:
for file in tqdm(predlist):
    if "_pred" in file:
        #get path name        
        path = (folder + file)
        #read in file
        data = pd.read_csv(path)
        #get element name, add to list
        element = data.columns[1]
        elem.append(element)
        # format columns
        data.columns = data.iloc[0]
        data = data.drop([0]).drop(data.columns[0], axis=1).astype(float)
        # remove predictions above 100 for majors
        if element in ['SiO2', 'MnO', 'Na2O']:
            data = data[data.Pred < 100]
        # remove all predictions below 0
        data = data[data.Pred > 0].reset_index(drop=True)
        # subselect relevant reference values
        ref = new_results[new_results.element == element].reset_index(drop=True)

#### COULD ADD CODE HERE TO CALCULATE AVERAGES, RMSE-PS, AND R2S FOR #####
####### ALL VALUES AND FOR THOSE ABOVE THE LOD BUT BELOW THE LOQ #########
        
        ###BRAGA###
        loq_b = ref['LOQ_Braga'].iloc[0]
        # select just predictions above the LOQ
        braga = data[data.Pred > loq_b].reset_index(drop=True)
        # get average concentration
        avg_b = braga['Actual'].mean()
        avg_braga.append(avg_b)
        # get R2
        r2_b = r2_score(braga.Actual, braga.Pred)
        r2_braga.append(r2_b)
        # get RMSE-P
        braga['sqerror'] = (braga.Actual - braga.Pred).pow(2)
        rmse_b = braga['sqerror'].mean() ** 0.5
        rmsep_braga.append(rmse_b)
        
        ##METALS###
        loq_m = ref['LOQ_metals'].iloc[0]
        # select just predictions above the LOQ
        metal = data[data.Pred > loq_m].reset_index(drop=True)
        # get average concentration
        avg_m = metal['Actual'].mean()
        avg_metal.append(avg_m)
        # get R2
        r2_m = r2_score(metal.Actual, metal.Pred)
        r2_metal.append(r2_m)
        # get RMSE-P
        metal['sqerror'] = (metal.Actual - metal.Pred).pow(2)
        rmse_m = metal['sqerror'].mean() ** 0.5
        rmsep_metal.append(rmse_m)

  0%|          | 0/28 [00:00<?, ?it/s]

#### Show results

In [9]:
instr = [instrument] * len(elem)
atmos = [atm] * len(elem)
num = [num_range] * len(elem)

temp = pd.DataFrame({
    "element" : elem,
    'instrument' : instr,
    'atmosphere' : atmos,
    'num_range' : num,
    "Avg_Braga" : avg_braga,
    "Avg_metals" : avg_metal,
    "RMSEP_Braga" : rmsep_braga,
    "RMSEP_metals" : rmsep_metal,
    "R2_Braga" : r2_braga,
    "R2_metals" : r2_metal
})

display(temp)

,element,instrument,atmosphere,num_range,Avg_Braga,Avg_metals,RMSEP_Braga,RMSEP_metals,R2_Braga,R2_metals
0,Li,LANL,Mars,250-1000,74.982667,45.318175,8.848230,8.305134,0.972080,0.957549
1,MnO,LANL,Mars,250-1000,0.368949,0.172405,0.339914,0.146782,0.526972,0.569041
2,Na2O,LANL,Mars,250-1000,2.612949,2.375119,0.556288,0.531150,0.850477,0.878852
3,Ni,LANL,Mars,250-1000,76.975230,76.975230,66.039062,66.039062,0.210709,0.210709
4,Pb,LANL,Mars,250-1000,16.261339,16.174956,11.940771,11.901814,0.208756,0.210586
5,Rb,LANL,Mars,250-1000,92.097213,90.128480,39.030775,38.584039,0.687352,0.697074
6,SiO2,LANL,Mars,250-1000,57.342051,57.342051,2.855192,2.855192,0.949434,0.949434
7,Sr,LANL,Mars,250-1000,466.166667,284.777647,52.437566,42.487355,0.793037,0.909333
8,Zn,LANL,Mars,250-1000,91.327122,90.511955,52.281691,51.997454,0.353613,0.362004


### Add to results table

In [10]:
full_table = full_table.append(temp).reset_index(drop = True)
display(full_table)

,element,instrument,atmosphere,num_range,Avg_Braga,Avg_metals,RMSEP_Braga,RMSEP_metals,R2_Braga,R2_metals
0,Li,LANL,Mars,250-1000,74.982667,45.318175,8.848230,8.305134,0.972080,0.957549
1,MnO,LANL,Mars,250-1000,0.368949,0.172405,0.339914,0.146782,0.526972,0.569041
2,Na2O,LANL,Mars,250-1000,2.612949,2.375119,0.556288,0.531150,0.850477,0.878852
3,Ni,LANL,Mars,250-1000,76.975230,76.975230,66.039062,66.039062,0.210709,0.210709
4,Pb,LANL,Mars,250-1000,16.261339,16.174956,11.940771,11.901814,0.208756,0.210586
5,Rb,LANL,Mars,250-1000,92.097213,90.128480,39.030775,38.584039,0.687352,0.697074
6,SiO2,LANL,Mars,250-1000,57.342051,57.342051,2.855192,2.855192,0.949434,0.949434
7,Sr,LANL,Mars,250-1000,466.166667,284.777647,52.437566,42.487355,0.793037,0.909333
8,Zn,LANL,Mars,250-1000,91.327122,90.511955,52.281691,51.997454,0.353613,0.362004


### Export full table

In [95]:
epath = "C:\\Users\\ytsma22c\\Google Drive\\Misc Work\\LOD paper\\test_calcs.csv"
full_table.to_csv(epath, index=False)